In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccposd51',
    proc_date,
    'temp_sap_vcdposd07'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdposd07 
(
	hash_id BIGINT
	,reqtsn STRING
	,datapakid STRING
	,record INT
	,recordmode STRING
	,calday DATE
	,weekday1 STRING
	,calweek STRING
	,calmonth STRING
	,calyear STRING
	,calmonth2 STRING
	,calquart1 STRING
	,halfyear1 STRING
	,fiscyear STRING
	,fiscvarnt STRING
	,fiscper STRING
	,fiscper3 STRING
	,loc_currcy STRING
	,rt_locatio STRING
	,segment STRING
	,plant STRING
	,bic_zc_loypg STRING
	,bic_zc_loynbr STRING
	,bic_zc_vinid STRING
	,rpa_wid STRING
	,rpa_tnr STRING
	,rpa_ttc STRING
	,bic_zk_loypnt INT
	,rpa_slc FLOAT
	,proc_date DATE
	,file_creation_ts TIMESTAMP
)
 TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW sap_vcdposd07 AS 
select 
    farm_fingerprint(CONCAT(
                ifnull(CAST(recordmode AS STRING), ""),
                ifnull(CAST(calday AS STRING), ""),
                ifnull(CAST(weekday1 AS STRING), ""),
                ifnull(CAST(calweek AS STRING), ""),
                ifnull(CAST(calmonth AS STRING), ""),
                ifnull(CAST(calyear AS STRING), ""),
                ifnull(CAST(fiscvarnt AS STRING), ""),
                ifnull(CAST(fiscper AS STRING), ""),
                ifnull(CAST(calmonth2 AS STRING), ""),
                ifnull(CAST(calquart1 AS STRING), ""),
                ifnull(CAST(halfyear1 AS STRING), ""),
                ifnull(CAST(fiscyear AS STRING), ""),
                ifnull(CAST(fiscper3 AS STRING), ""),
                ifnull(CAST(loc_currcy AS STRING), ""),
                ifnull(CAST(segment AS STRING), ""),
                ifnull(CAST(plant AS STRING), ""),
                ifnull(CAST(rt_locatio AS STRING), ""),
                ifnull(CAST(_bic_zc_loypg AS STRING), ""),
                ifnull(CAST(_bic_zc_loynbr AS STRING), ""),
                ifnull(CAST(_bic_zc_vinid AS STRING), ""),
                ifnull(CAST(rpa_wid AS STRING), ""),
                ifnull(CAST(rpa_tnr AS STRING), ""),
                ifnull(CAST(rpa_ttc AS STRING), "")
            )) hash_id,
            reqtsn,
            datapakid,
            record,
            recordmode,
            CASE WHEN calday = '00000000' THEN NULL ELSE to_date(calday, 'yyyyMMdd') END AS calday,
            weekday1,
            calweek,
            calmonth,
            calyear,
            calmonth2,
            calquart1,
            halfyear1,
            fiscyear,
            CASE WHEN fiscvarnt = '' THEN NULL ELSE fiscvarnt END AS fiscvarnt,
            CASE WHEN fiscper = '' THEN NULL ELSE fiscper END AS fiscper,
            CASE WHEN fiscper3 = '' THEN NULL ELSE fiscper3 END AS fiscper3,
            CASE WHEN loc_currcy = '' THEN NULL ELSE loc_currcy END AS loc_currcy,
            CASE WHEN rt_locatio = '' THEN NULL ELSE rt_locatio END AS rt_locatio,
            LTRIM('0', segment) AS segment,
            CASE WHEN plant = '' THEN NULL ELSE plant END AS plant,
            CASE WHEN _bic_zc_loypg = '' THEN NULL ELSE _bic_zc_loypg END AS bic_zc_loypg,
            CASE WHEN _bic_zc_loynbr = '' THEN NULL ELSE _bic_zc_loynbr END AS bic_zc_loynbr,
            CASE WHEN _bic_zc_vinid = '' THEN NULL ELSE _bic_zc_vinid END AS bic_zc_vinid,
            CASE WHEN rpa_wid = '' THEN NULL ELSE rpa_wid END AS rpa_wid,
            CASE WHEN rpa_tnr = '' THEN NULL ELSE rpa_tnr END AS rpa_tnr,
            CASE WHEN rpa_ttc = '' THEN NULL ELSE rpa_ttc END AS rpa_ttc,
            _bic_zk_loypnt AS bic_zk_loypnt,
            rpa_slc * 100 AS rpa_slc,
            proc_date AS proc_date,
            file_creation_ts
        from (
                select * except(rn) from (
                    select *, ROW_NUMBER() OVER (PARTITION BY reqtsn, datapakid, record ORDER BY proc_date DESC) rn from temp_sap_vcdposd07
                ) a where rn = 1 
            ) a
""")


In [0]:
spark.sql(f"""
    MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdposd07 main_tbl
    USING sap_vcdposd07 temp_tbl 
    ON main_tbl.hash_id = temp_tbl.hash_id
        AND main_tbl.reqtsn = temp_tbl.reqtsn 
        AND main_tbl.datapakid = temp_tbl.datapakid
        AND main_tbl.record = temp_tbl.record
    WHEN NOT MATCHED THEN INSERT *;
""")